In [59]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from geopy.geocoders import Nominatim
from tqdm import tqdm
from geopy.distance import geodesic
import folium
from folium.plugins import MarkerCluster


In [66]:
df = pd.read_csv(r'Data_bomen/Stom_Data_Incidents.csv')
df = df[df['Damage_Type'] == 'Tree']
df

,Incident_ID,Date,Incident_Starttime,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,Damage_Type,LON,LAT
0,511,2018-09-07,08:25:18,10:00:56,01:35:38,2,Amstelveen,Amstelveen,Tree,4.838685,52.281552
1,738,2018-09-10,16:46:38,18:00:55,01:14:17,2,Victor,Amsterdam,Tree,4.930968,52.359724
4,1493,2018-09-21,06:59:05,08:21:25,01:22:20,2,Amstelveen,Amstelveen,Tree,4.879741,52.301365
6,1502,2018-09-21,08:29:22,10:06:43,01:37:21,2,Dirk,Amsterdam,Tree,4.870656,52.355189
7,1507,2018-09-21,09:25:55,11:43:54,02:17:59,2,Teunis,Amsterdam,Tree,4.832333,52.380748
...,...,...,...,...,...,...,...,...,...,...,...
989,293390,2018-01-18,10:25:52,10:37:57,00:12:05,2,Uithoorn,Uithoorn,Tree,4.828782,52.238112
994,293407,2018-01-18,10:34:12,11:21:49,00:47:37,1,Teunis,Amsterdam,Tree,4.858053,52.375563
995,293411,2018-01-18,10:31:10,11:20:21,00:49:11,2,Victor,Amsterdam,Tree,4.928020,52.369366
996,293412,2018-01-18,10:31:54,12:59:04,02:27:10,1,Aalsmeer,Aalsmeer,Tree,4.780278,52.254294


In [43]:
incidenten = pd.read_csv(r'Stormdata & FireStations geadresseerd.csv')

In [44]:
incidenten_sorted = incidenten[incidenten['Damage_Type'] == 'Tree']
incidenten_sorted = incidenten_sorted.drop('Unnamed: 0', axis=1)

In [57]:
incidenten_sorted

,Incident_ID,Incident_Starttime,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Damage_Type,Municipality,LON,LAT,Day,Month,Year,City,Suburb,Residential,Road,Zipcode
0,511,08:25:18,10:00:56,95.633,2.0,Amstelveen,Tree,Amstelveen,4.838685,52.281552,7,9,2018,NaN,NaN,NaN,Bouwerij,1185 XW
1,738,16:46:38,18:00:55,74.283,2.0,Victor,Tree,Amsterdam,4.930968,52.359724,10,9,2018,Amsterdam,Oost,Dapperbuurt,Linnaeusplantsoen,1093 KL
4,1493,06:59:05,08:21:25,82.333,2.0,Amstelveen,Tree,Amstelveen,4.879741,52.301365,21,9,2018,NaN,NaN,NaN,Oranjebaan,1183 PL
5,1502,08:29:22,10:06:43,97.350,2.0,Dirk,Tree,Amsterdam,4.870656,52.355189,21,9,2018,Amsterdam,Zuid,Museumkwartier,Van Breestraat,1071 ZM
6,1507,09:25:55,11:43:54,137.983,2.0,Teunis,Tree,Amsterdam,4.832333,52.380748,21,9,2018,Amsterdam,Nieuw-West,NaN,M.C. Addicksstraat,1063 VZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5276,436479,18:47:00,20:36:09,109.150,5.0,Victor,Tree,Amsterdam,4.944491,52.352064,31,7,2023,Amsterdam,Oost,Middenmeer,Rutherfordstraat,1098 TM
5277,436603,14:10:34,15:13:30,62.933,2.0,Osdorp,Tree,Amsterdam,4.824533,52.376163,2,8,2023,Amsterdam,Nieuw-West,NaN,Louis Couperusstraat,1064 CE
5282,436834,03:38:27,06:29:11,170.733,5.0,Nico,Tree,Amsterdam,4.928197,52.390568,6,8,2023,Amsterdam,Noord,Tuindorp Buiksloot,Nieuwendammerdijk,1025 LP
5283,436904,21:52:41,22:16:29,23.800,2.0,Nico,Tree,Amsterdam,4.917106,52.368343,6,8,2023,Amsterdam,Centrum,NaN,Hoogte Kadijk,1018 BV


In [60]:
# Create a map centered around Amsterdam
amsterdam_map = folium.Map(location=[52.3676, 4.9041], zoom_start=12)

# Create a MarkerCluster to add incidents to the map
marker_cluster = MarkerCluster().add_to(amsterdam_map)

# Add markers for each incident
for index, incident in incidenten_sorted.iterrows():
    # Extract incident details
    incident_id = incident['Incident_ID']
    lat = incident['LAT']
    lon = incident['LON']
    damage_type = incident['Damage_Type']
    priority = incident['Incident_Priority']

    # Create a popup for the marker
    popup_text = f"Incident ID: {incident_id}<br>Damage Type: {damage_type}<br>Priority: {priority}"

    # Add marker to the cluster
    folium.Marker(location=[lat, lon], popup=popup_text).add_to(marker_cluster)

# Save the map as an HTML file
amsterdam_map.save("incident_map.html")


In [46]:
trees = pd.read_csv(r'BOMEN_DATA_WITH_ZIP.csv')
trees = trees.drop('Unnamed: 0', axis=1)
trees.head()

,OBJECTNUMMER,id,soortnaam,soortnaamKort,SoortnaamNL,soortnaamTop,boomhoogteklasseActueel,stamdiameterklasse,jaarVanAanleg,typeObject,...,typeBeheerderPlus,typeEigenaarPlus,SDVIEW,RADIUS,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Zipcode,Zip4
0,1,919934,Ulmus hollandica `Vegeta`,Ulmus,Huntingdon-iep,Iep (Ulmus),b. 6 tot 9 m.,NaN,1978,Boom niet vrij uitgroeiend,...,Stadsdeel Zuid,Gemeente Amsterdam,K,2,POINT(4.9026919 52.340093),POINT(52.340093 4.9026919),4.902692,52.340093,1185 XW,1185
1,2,919935,Fraxinus excelsior `Westhof`s Glorie`,Fraxinus,Gewone es (cultuurvariëteit),Es (Fraxinus),c. 9 tot 12 m.,"0,2 tot 0,3 m.",1990,Boom niet vrij uitgroeiend,...,Stadsdeel Zuid,Gemeente Amsterdam,K,3,POINT(4.8552081 52.331984),POINT(52.331984 4.8552081),4.855208,52.331984,1185 XW,1185
2,3,919936,Ulmus glabra `Lutescens`,Ulmus,Gele bergiep (cultuurvariëteit),Iep (Ulmus),b. 6 tot 9 m.,NaN,2002,Boom niet vrij uitgroeiend,...,Stadsdeel Zuid,Gemeente Amsterdam,K,2,POINT(4.9036699 52.348837),POINT(52.348837 4.9036699),4.903670,52.348837,1185 XW,1185
3,4,919937,Quercus robur,Quercus,Zomereik,Eik (Quercus),b. 6 tot 9 m.,NaN,1985,Boom niet vrij uitgroeiend,...,Stadsdeel Zuid,Gemeente Amsterdam,K,2,POINT(4.875886 52.341056),POINT(52.341056 4.875886),4.875886,52.341056,1185 XW,1185
4,5,919939,Styphnolobium japonicum,Overig,Honingboom,Styphnolobium,a. tot 6 m.,NaN,2002,Boom niet vrij uitgroeiend,...,Stadsdeel Zuid,Gemeente Amsterdam,K,1,POINT(4.8803689 52.332891),POINT(52.332891 4.8803689),4.880369,52.332891,1185 XW,1185


In [58]:
# # Functie om de afstand tussen twee punten te berekenen
# def calculate_distance(lat1, lon1, lat2, lon2):
#     return geodesic((lat1, lon1), (lat2, lon2)).meters

# # Voor elk incident, vind de dichtstbijzijnde boom
# for index, incident in tqdm(incidenten_sorted.iterrows(), total=len(incidenten_sorted)):
#     incident_lat = incident['LAT']
#     incident_lon = incident['LON']

#     # Bereken afstand tot alle bomen en selecteer de dichtstbijzijnde boom
#     trees['Afstand'] = trees.apply(lambda row: calculate_distance(incident_lat, incident_lon, row['LAT'], row['LNG']), axis=1)
    
#     # Check if there are rows in trees DataFrame
#     if not trees.empty:
#         dichtstbijzijnde_boom = trees.loc[trees['Afstand'].idxmin()]

#         # Check if 'Boom_Type' is in the dichtstbijzijnde_boom DataFrame
#         if 'Boom_Type' in dichtstbijzijnde_boom:
#             # Voeg het type boom toe aan de incident DataFrame
#             incidenten_sorted.at[index, 'Boom_Type'] = dichtstbijzijnde_boom['Boom_Type']
